# 1. Thư viện

In [1]:
import os
import json
import re

from dotenv import load_dotenv
from langchain_qdrant import Qdrant, RetrievalMode
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from apikeys_GEMINI import APIKeyManager

from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
load_dotenv()

MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
if MODEL_GEMINI is None:
    raise ValueError("Environment variable MODEL_GEMINI is not set")
elif not MODEL_GEMINI.startswith("models/"):
    MODEL_GEMINI = f"models/{MODEL_GEMINI}"

APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)

URL_QDRANT_2 = os.getenv("URL_QDRANT_2")
API_QDRANT_2 = os.getenv("API_QDRANT_2")

MAX_DOCS_FOR_CONTENT = 5

# 2. Mô hình embedding

In [3]:
EMBEDDINGS_MODEL_bkai = "bkai-foundation-models/vietnamese-bi-encoder"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings_bkai = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL_bkai,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 3. Kết nối đến Collection

In [4]:
exist_Qdrant_More_Keywords = Qdrant.from_existing_collection(
    embedding = embeddings_bkai,
    url = URL_QDRANT_2,
    api_key = API_QDRANT_2,
    prefer_grpc=True,
    collection_name = "semantic_Luat_bkai_More_Keywords",
	metadata_payload_key="metadata"
)

# 4. Hàm sinh ra Similarity Queries

In [5]:
def query_generator(original_query: str, key_manager) -> list[str]:
    """Generate queries from original query"""
    # Câu truy vấn gốc
    query = original_query
    
    # Cập nhật prompt để yêu cầu rõ ràng chỉ trả về 3 câu truy vấn và câu gốc
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Bạn là một trợ lý hữu ích, có nhiệm vụ tạo ra nhiều truy vấn tìm kiếm dựa trên một truy vấn gốc."),
            ("human", "Tạo 3 truy vấn tìm kiếm liên quan đến: {original_query}. Mỗi truy vấn trên một dòng mới, và đảm bảo chỉ trả về các truy vấn, không có thêm bất kỳ giải thích hay văn bản nào khác."),
        ]
    )
    
    model = ChatGoogleGenerativeAI(
        # key api google gemini, nếu test mà bị báo lỗi api core thì lấy api khác trong .env để test
        google_api_key=key_manager.get_next_key(),
        model=MODEL_GEMINI,
        temperature=0.1
    )
    
    query_generator_chain = (
        prompt | model | StrOutputParser()
    )
    
    # Kết quả sẽ là một chuỗi các câu truy vấn cách nhau bằng dấu xuống dòng
    result = query_generator_chain.invoke({"original_query": query})
    
    # Tách kết quả thành danh sách các câu truy vấn
    generated_queries = result.strip().split('\n')
    
    # Đảm bảo chỉ lấy 3 câu nếu có nhiều hơn 3 câu truy vấn sinh ra
    if len(generated_queries) > 3:
        generated_queries = generated_queries[:3]
    
    # Kết hợp câu gốc với các câu truy vấn sinh ra
    queries = [query] + generated_queries
    
    return queries

# 5. Hàm Re-rank RRF

In [6]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (reciprocal rank fusion)"""
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTENT]]

# 6. Tách Keywords

## 6.1. Tách keywords

In [7]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)  # Loại bỏ các dấu câu không mong muốn
    words = text.split()                   # Tách các từ theo khoảng trắng
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

## 6.2. Xử lý lower case

In [8]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

# 7. Truy vấn

In [9]:
def print_Results_MoreKeywords(results):
    for doc,score in results:
        print("Page_Content:\n",doc.page_content)
        # print("Context:\n",doc.metadata['context'],"\n")
        print("Stt:",doc.metadata['stt'])
        print("Số hiệu:",doc.metadata['so_hieu'])
        print("Chủ đề:",doc.metadata['chu_de'])
        print("Chương:",doc.metadata["Chapter"])
        print("Mục:",doc.metadata["Section"])
        print("Tiểu mục:",doc.metadata["Mini-Section"])
        print("Điều:",doc.metadata["Article"])
        print("Khoản:",doc.metadata["Article-Section"])
        print("Score:",score,"\n","-----------------------")

## 7.1. Tạo Filter

In [10]:
def create_should_filter(user_keywords, metadata_fields):
    should_conditions = []

    for keyword in user_keywords:
        for field in metadata_fields:
            should_conditions.append(FieldCondition(
                key=field, 
                match=MatchValue(value=keyword)
            ))

    # Trả về bộ lọc với các điều kiện `should`
    return Filter(
        should=should_conditions
    )

## 7.2. Truy vấn với Filter

In [11]:
def search_documents_with_should_filter(user_query, metadata_fields, top_k=5):
    # Tách keywords từ query của user
    user_keywords = process_keywords(extract_keywords(user_query))
    
    # Tạo bộ lọc `should`
    filter_conditions = create_should_filter(user_keywords, metadata_fields)
    
    # Thực hiện tìm kiếm trên Qdrant với filter `should`
    search_results = exist_Qdrant_More_Keywords.similarity_search_with_score(
        query=user_query,
        filter=filter_conditions,
        k=top_k
    )
    
    return search_results

In [12]:
def combined_search_with_rerank(user_query: str, key_manager, metadata_fields, top_k=5):
    # Gọi hàm query_generator để sinh ra 3 truy vấn từ query gốc
    queries = query_generator(user_query, key_manager)

    # Lưu trữ kết quả cho từng query
    query_results = []

    # Thực hiện tìm kiếm cho mỗi query trong danh sách queries
    for query in queries:
        search_results = search_documents_with_should_filter(query, metadata_fields, top_k=top_k)
        query_results.append(search_results)  # Lưu kết quả riêng cho từng query

    # Sử dụng hàm reciprocal_rank_fusion để re-rank kết quả từ nhiều query
    reranked_results = reciprocal_rank_fusion(query_results)

    # Lấy top 5 kết quả sau khi re-rank
    top_results = reranked_results[:top_k]

    return top_results

In [13]:
# Các metadata fields cần lọc
metadata_fields = [
    "metadata.loai_van_ban_Keywords", 
    "metadata.noi_ban_hanh_Keywords", 
    "metadata.so_hieu", 
    "metadata.linhvuc_nganh_Keywords", 
    "metadata.ngay_ban_hanh", 
    "metadata.ngay_hieu_luc", 
    "metadata.chu_de_Keywords", 
    "metadata.key_words", 
    "metadata.Chapter_Keywords", 
    "metadata.Section_Keywords",  
    "metadata.Mini-Section_Keywords", 
    "metadata.Article_Keywords", 
    "metadata.Content_Keywords", 
    "metadata.Article-Section_Keywords"
]

In [15]:

# Ví dụ sử dụng
user_query= "Các quyền lợi của người tham gia bảo hiểm xã hội là gì?"

results = combined_search_with_rerank(user_query, key_manager, metadata_fields, top_k=5)

# In ra kết quả top 5 tài liệu
print_Results_MoreKeywords(results)


Page_Content:
 Điều 10. Quyền của người tham gia và người thụ hưởng chế độ bảo hiểm xã hội
1. Người tham gia bảo hiểm xã hội có các quyền sau đây:
a) Hưởng chế độ bảo hiểm xã hội theo quy định của Luật này;
b) Được cấp sổ bảo hiểm xã hội;
c) Được cơ quan bảo hiểm xã hội định kỳ hằng tháng cung cấp thông tin về việc đóng bảo hiểm xã hội thông qua phương tiện điện tử; được cơ quan bảo hiểm xã hội xác nhận thông tin về đóng bảo hiểm xã hội khi có yêu cầu;
d) Yêu cầu người sử dụng lao động và các cơ quan, tổ chức có liên quan thực hiện đầy đủ trách nhiệm về bảo hiểm xã hội đối với mình theo quy định của pháp luật;
đ) Được tuyên truyền, phổ biến chính sách, pháp luật về bảo hiểm xã hội;
e) Chủ động đi khám giám định y khoa để xác định mức suy giảm khả năng lao động nếu thuộc trường hợp đang bảo lưu thời gian đóng bảo hiểm xã hội và được thanh toán phí giám định y khoa khi kết quả giám định y khoa đủ điều kiện để hưởng chế độ bảo hiểm xã hội theo quy định của Luật này;
g) Khiếu nại, tố cáo v